In [644]:
import requests
import pandas as pd
import sqlite3
import time

In [815]:
#Para sacar el id de las ligas y guardarlos en la BBDD

url = "https://api-football-v1.p.rapidapi.com/v3/leagues"
countries = ['Spain','England','Italy','Germany','France', 'Portugal', 'Netherlands']
id_liga = []
nombre_liga = []
country_name = []
logo_liga = []

for country in countries:
    
    querystring = {"country":country}
    headers = {
        "X-RapidAPI-Key": "b78f5cd68cmsh17c6227d104p1755afjsn96dc33575295",
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring).json()

    for a in range(len(response)):
        id_liga.append(response['response'][a]['league']['id'])
        nombre_liga.append(response['response'][a]['league']['name'])
        country_name.append(country)
        logo_liga.append(response['response'][a]['league']['logo'])

    time.sleep(0.5)




In [646]:
#Creo la BBDD

conn = sqlite3.connect('BBDD_futbol.sqlite')

In [816]:
#Creamos la BBDD y añadimos la información de las ligas

ligas_futbol = pd.DataFrame({'id_liga': id_liga, 
                             'nombre_liga':nombre_liga, 
                             'country': country_name,
                            'logo_liga':logo_liga})

ligas_futbol.to_sql('tabla_ligas', conn, if_exists='replace')

42

In [648]:
#Recojo la información de los equipos para las ligas top escogidas (primera y segunda de las 5 grandes ligas de Europa)

url = "https://api-football-v1.p.rapidapi.com/v3/teams"

id_ligas_top = ['140','39','135','78','61','94','88']
id_equipo = []
nombre_equipo = []
liga_id = []
estadio_equipo = []
logo_equipo = []


for liga in id_ligas_top:
    
    querystring = {"league": liga,
                  'season': '2022'}

    headers = {
        "X-RapidAPI-Key": "b78f5cd68cmsh17c6227d104p1755afjsn96dc33575295",
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring).json()
    
    for a in range(len(response['response'])):
        id_equipo.append(response['response'][a]['team']['id'])
        nombre_equipo.append(response['response'][a]['team']['name'])
        liga_id.append(response['parameters']['league'])
        logo_equipo.append(response['response'][a]['team']['logo'])
        estadio_equipo.append(response['response'][a]['venue']['name'])
        
    time.sleep(0.5)

In [649]:
#Creo el DataFrame y lo añado a la BBDD
info_equipos = pd.DataFrame({'id_liga': liga_id,  
                             'id_equipo':id_equipo, 
                             'nombre_equipo':nombre_equipo,
                             'estadio' :estadio_equipo,
                             'logo_equipo':logo_equipo
                            })

info_equipos.to_sql('info_equipos', conn, if_exists='replace')

134

In [671]:
#Saco el fixture id, partidos de local, visitante, resultado y arbitro de cada liga y otros datos de interés

jornada = []
temporada = []
liga = []
equipo_local = []
id_equipo_local = []
equipo_visitante = []
id_equipo_visitante = []
goles_local = []
goles_visitante = []
arbitro = []
fixture_id = []
goles_descanso_local = []
goles_descanso_visitante = []
estadio = []
fecha = []


for x in set(liga_id):
    url = "https://api-football-v1.p.rapidapi.com/v3/fixtures"

    querystring = {"league":x,"season":"2022"}

    headers = {
        "X-RapidAPI-Key": "b78f5cd68cmsh17c6227d104p1755afjsn96dc33575295",
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
    }
    

    response = requests.request("GET", url, headers=headers, params=querystring).json()
    
    for a in range(len(response['response'])):
        if response['response'][a]['fixture']['status']['long'] == 'Match Finished':
            equipo_local.append(response['response'][a]['teams']['home']['name'])
            id_equipo_local.append(response['response'][a]['teams']['home']['id'])
            equipo_visitante.append(response['response'][a]['teams']['away']['name'])
            id_equipo_visitante.append(response['response'][a]['teams']['away']['id'])
            goles_local.append(response['response'][a]['goals']['home'])
            goles_visitante.append(response['response'][a]['goals']['away'])
            arbitro.append(response['response'][a]['fixture']['referee'])
            fixture_id.append(response['response'][a]['fixture']['id'])
            goles_descanso_local.append(response['response'][a]['score']['halftime']['home'])
            goles_descanso_visitante.append(response['response'][a]['score']['halftime']['away'])
            estadio.append(response['response'][a]['fixture']['venue']['name'])
            liga.append(x)
            jornada.append(response['response'][a]['league']['round'])
            temporada.append(response['response'][a]['league']['season'])
            fecha.append(response['response'][a]['fixture']['date'])
    


In [673]:
#Creo el dataframe con la información principal de los partidos

matches_info = pd.DataFrame({'id_liga':liga,
                             'temporada':temporada,
                             'jornada':jornada,
                             'fecha':fecha,
                            'fixture_id': fixture_id,
                            'estadio':estadio,
                            'arbitro':arbitro,
                            'id_equipo_local':id_equipo_local,
                            'nombre_equipo_local':equipo_local,
                            'id_equipo_visitante':id_equipo_visitante,
                            'nombre_equipo_visitante': equipo_visitante
                           })

matches_info.to_sql('info_partidos', conn, if_exists='replace')

964

In [869]:
#Utilizo una query para sacar el fixture id de cada partido y obtener asi las estadisticas
df_matches_info = pd.read_sql('SELECT fixture_id, jornada FROM info_partidos', conn)


In [874]:
#Convierto la columna en lista para poder iterar mas facilmente
fixture_id = df_matches_info['fixture_id'].tolist()
jornada_fx = df_matches_info['jornada'].tolist()

In [981]:
#Saco las estadisticas por partido por cada fixture_id (identificador del partido)

url = "https://api-football-v1.p.rapidapi.com/v3/fixtures"

fx_id = []

#Estadisticas por equipo por partido equipo
jornada = []
condicion = []
goles_a_favor = []
id_equipo = []
nombre_team = []
goles_a_favor_descanso = []
shots_on_goal = []
shots_off_goal = []
total_shots=[]
blocked_shots = []
shots_insidebox = []
shots_outsidebox=[]
fouls=[]
corners=[]
offsides = []
ball_possession=[]
yellow_cards=[]
red_cards=[]
goalkeeper_saves=[]
pass_precision=[]
puntos = []

#Estadisticas por partido  equipo rival
id_equipo_rival = []
goles_recibidos = []
goles_recibidos_descanso = []
puntos_perdidos = []
shots_on_goal_rival = []
shots_off_goal_rival = []
total_shots_rival=[]
blocked_shots_rival = []
shots_insidebox_rival = []
shots_outsidebox_rival=[]
fouls_rival=[]
corners_rival=[]
offsides_rival = []
ball_possession_rival=[]
yellow_cards_rival=[]
red_cards_rival=[]
goalkeeper_saves_rival=[]
pass_precision_rival=[]

#Estadisticas generales partidos
fixture_id_partido = []
goles_local =[]
goles_away=[]
id_local= []
id_away=[]
nombre_team_local=[]
nombre_team_away=[]
goles_local_descanso=[]
goles_away_descanso=[]
shots_on_goal_local=[]
shots_on_goal_away=[]
shots_off_goal_local=[]
shots_off_goal_away=[]
total_shots_local=[]
total_shots_away=[]
blocked_shots_local=[]
blocked_shots_away=[]
shots_insidebox_local=[]
shots_insidebox_away=[]
shots_outsidebox_local=[]
shots_outsidebox_away=[]
fouls_local=[]
fouls_away=[]
corners_local=[]
corners_away=[]
offsides_local=[]
offsides_away=[]
ball_possession_local=[]
ball_possession_away=[]
yellow_cards_local=[]
yellow_cards_away=[]
red_cards_local=[]
red_cards_away=[]
goalkeeper_saves_local=[]
goalkeeper_saves_away=[]
pass_precision_local=[]
pass_precision_away=[]
logo_local = []
logo_away = []

#variable flag
flag = 0

for fixture in fixture_id:
    
    querystring = {"id":fixture}

    headers = {
        "X-RapidAPI-Key": "b78f5cd68cmsh17c6227d104p1755afjsn96dc33575295",
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring).json()

    for a in range(len(response['response'])):
        fx_id.append(fixture)
        fx_id.append(fixture)
                
        #Estadisticas por equipo por jornada
        goles_a_favor.append(response['response'][a]['score']['fulltime']['home'])
        goles_a_favor.append(response['response'][a]['score']['fulltime']['away'])
        id_equipo.append(response['response'][a]['statistics'][0]['team']['id'])
        id_equipo.append(response['response'][a]['statistics'][1]['team']['id'])
        nombre_team.append(response['response'][a]['statistics'][0]['team']['name'])
        nombre_team.append(response['response'][a]['statistics'][1]['team']['name'])
        goles_a_favor_descanso.append(response['response'][a]['score']['halftime']['home'])
        goles_a_favor_descanso.append(response['response'][a]['score']['halftime']['away'])
        shots_on_goal.append(response['response'][a]['statistics'][0]['statistics'][0]['value'])
        shots_on_goal.append(response['response'][a]['statistics'][1]['statistics'][0]['value'])
        shots_off_goal.append(response['response'][a]['statistics'][0]['statistics'][1]['value'])
        shots_off_goal.append(response['response'][a]['statistics'][1]['statistics'][1]['value'])
        total_shots.append(response['response'][a]['statistics'][0]['statistics'][2]['value'])
        total_shots.append(response['response'][a]['statistics'][1]['statistics'][2]['value'])
        blocked_shots.append(response['response'][a]['statistics'][0]['statistics'][3]['value'])
        blocked_shots.append(response['response'][a]['statistics'][1]['statistics'][3]['value'])
        shots_insidebox.append(response['response'][a]['statistics'][0]['statistics'][4]['value'])
        shots_insidebox.append(response['response'][a]['statistics'][1]['statistics'][4]['value'])
        shots_outsidebox.append(response['response'][a]['statistics'][0]['statistics'][5]['value'])
        shots_outsidebox.append(response['response'][a]['statistics'][1]['statistics'][5]['value'])
        fouls.append(response['response'][a]['statistics'][0]['statistics'][6]['value'])
        fouls.append(response['response'][a]['statistics'][1]['statistics'][6]['value'])
        corners.append(response['response'][a]['statistics'][0]['statistics'][7]['value'])
        corners.append(response['response'][a]['statistics'][1]['statistics'][7]['value'])
        offsides.append(response['response'][a]['statistics'][0]['statistics'][8]['value'])
        offsides.append(response['response'][a]['statistics'][1]['statistics'][8]['value'])
        ball_possession.append(response['response'][a]['statistics'][0]['statistics'][9]['value'])
        ball_possession.append(response['response'][a]['statistics'][1]['statistics'][9]['value'])
        yellow_cards.append(response['response'][a]['statistics'][0]['statistics'][10]['value'])
        yellow_cards.append(response['response'][a]['statistics'][1]['statistics'][10]['value'])
        red_cards.append(response['response'][a]['statistics'][0]['statistics'][11]['value'])
        red_cards.append(response['response'][a]['statistics'][1]['statistics'][11]['value'])
        goalkeeper_saves.append(response['response'][a]['statistics'][0]['statistics'][12]['value'])
        goalkeeper_saves.append(response['response'][a]['statistics'][1]['statistics'][12]['value'])
        pass_precision.append(response['response'][a]['statistics'][0]['statistics'][13]['value'])
        pass_precision.append(response['response'][a]['statistics'][1]['statistics'][13]['value'])
        
        #Estadisticas equipo rival (analizar datos en contra de cada equipo)
        goles_recibidos.append(response['response'][a]['score']['fulltime']['away'])
        goles_recibidos.append(response['response'][a]['score']['fulltime']['home'])
        id_equipo_rival.append(response['response'][a]['statistics'][1]['team']['id'])
        id_equipo_rival.append(response['response'][a]['statistics'][0]['team']['id'])
        goles_recibidos_descanso.append(response['response'][a]['score']['halftime']['away'])
        goles_recibidos_descanso.append(response['response'][a]['score']['halftime']['home'])
        shots_on_goal_rival.append(response['response'][a]['statistics'][1]['statistics'][0]['value'])
        shots_on_goal_rival.append(response['response'][a]['statistics'][0]['statistics'][0]['value'])
        shots_off_goal_rival.append(response['response'][a]['statistics'][1]['statistics'][1]['value'])
        shots_off_goal_rival.append(response['response'][a]['statistics'][0]['statistics'][1]['value'])
        total_shots_rival.append(response['response'][a]['statistics'][1]['statistics'][2]['value'])
        total_shots_rival.append(response['response'][a]['statistics'][0]['statistics'][2]['value'])
        blocked_shots_rival.append(response['response'][a]['statistics'][1]['statistics'][3]['value'])
        blocked_shots_rival.append(response['response'][a]['statistics'][0]['statistics'][3]['value'])
        shots_insidebox_rival.append(response['response'][a]['statistics'][1]['statistics'][4]['value'])
        shots_insidebox_rival.append(response['response'][a]['statistics'][0]['statistics'][4]['value'])
        shots_outsidebox_rival.append(response['response'][a]['statistics'][1]['statistics'][5]['value'])
        shots_outsidebox_rival.append(response['response'][a]['statistics'][0]['statistics'][5]['value'])
        fouls_rival.append(response['response'][a]['statistics'][1]['statistics'][6]['value'])
        fouls_rival.append(response['response'][a]['statistics'][0]['statistics'][6]['value'])
        corners_rival.append(response['response'][a]['statistics'][1]['statistics'][7]['value'])
        corners_rival.append(response['response'][a]['statistics'][0]['statistics'][7]['value'])
        offsides_rival.append(response['response'][a]['statistics'][1]['statistics'][8]['value'])
        offsides_rival.append(response['response'][a]['statistics'][0]['statistics'][8]['value'])
        ball_possession_rival.append(response['response'][a]['statistics'][1]['statistics'][9]['value'])
        ball_possession_rival.append(response['response'][a]['statistics'][0]['statistics'][9]['value'])
        yellow_cards_rival.append(response['response'][a]['statistics'][1]['statistics'][10]['value'])
        yellow_cards_rival.append(response['response'][a]['statistics'][0]['statistics'][10]['value'])
        red_cards_rival.append(response['response'][a]['statistics'][1]['statistics'][11]['value'])
        red_cards_rival.append(response['response'][a]['statistics'][0]['statistics'][11]['value'])
        goalkeeper_saves_rival.append(response['response'][a]['statistics'][1]['statistics'][12]['value'])
        goalkeeper_saves_rival.append(response['response'][a]['statistics'][0]['statistics'][12]['value'])
        pass_precision_rival.append(response['response'][a]['statistics'][1]['statistics'][13]['value'])
        pass_precision_rival.append(response['response'][a]['statistics'][0]['statistics'][13]['value'])
        jornada.append(jornada_fx[flag])
        jornada.append(jornada_fx[flag])
        condicion.append('local')
        condicion.append('visitante')
        
        #para saber los puntos conseguidos por jornada, añado los puntos a local y visitante en función del resultado del partido:

        if response['response'][a]['score']['fulltime']['home'] > response['response'][a]['score']['fulltime']['away']:

            puntos.append(3)
            puntos.append(0)

        elif response['response'][a]['score']['fulltime']['home'] == response['response'][a]['score']['fulltime']['away']:

            puntos.append(1)
            puntos.append(1)

        elif response['response'][a]['score']['fulltime']['home'] < response['response'][a]['score']['fulltime']['away']:

            puntos.append(0)
            puntos.append(3)
        
        #para saber los puntos perdidos por jornada, añado los puntos a local y visitante en función del resultado del partido:
        
        if response['response'][a]['score']['fulltime']['home'] > response['response'][a]['score']['fulltime']['away']:

            puntos_perdidos.append(0)
            puntos_perdidos.append(3)

        elif response['response'][a]['score']['fulltime']['home'] == response['response'][a]['score']['fulltime']['away']:

            puntos_perdidos.append(1)
            puntos_perdidos.append(1)

        elif response['response'][a]['score']['fulltime']['home'] < response['response'][a]['score']['fulltime']['away']:

            puntos_perdidos.append(3)
            puntos_perdidos.append(0)
            
        
        #Estadísticas por partido:
        fixture_id_partido.append(fixture)
        goles_local.append(response['response'][a]['score']['fulltime']['home'])
        goles_away.append(response['response'][a]['score']['fulltime']['away'])
        id_local.append(response['response'][a]['statistics'][0]['team']['id'])
        id_away.append(response['response'][a]['statistics'][1]['team']['id'])
        nombre_team_local.append(response['response'][a]['statistics'][0]['team']['name'])
        nombre_team_away.append(response['response'][a]['statistics'][1]['team']['name'])
        logo_local.append(response['response'][a]['statistics'][0]['team']['logo'])
        logo_away.append(response['response'][a]['statistics'][1]['team']['logo'])
        goles_local_descanso.append(response['response'][a]['score']['halftime']['home'])
        goles_away_descanso.append(response['response'][a]['score']['halftime']['away'])
        shots_on_goal_local.append(response['response'][a]['statistics'][0]['statistics'][0]['value'])
        shots_on_goal_away.append(response['response'][a]['statistics'][1]['statistics'][0]['value'])
        shots_off_goal_local.append(response['response'][a]['statistics'][0]['statistics'][1]['value'])
        shots_off_goal_away.append(response['response'][a]['statistics'][1]['statistics'][1]['value'])
        total_shots_local.append(response['response'][a]['statistics'][0]['statistics'][2]['value'])
        total_shots_away.append(response['response'][a]['statistics'][1]['statistics'][2]['value'])
        blocked_shots_local.append(response['response'][a]['statistics'][0]['statistics'][3]['value'])
        blocked_shots_away.append(response['response'][a]['statistics'][1]['statistics'][3]['value'])
        shots_insidebox_local.append(response['response'][a]['statistics'][0]['statistics'][4]['value'])
        shots_insidebox_away.append(response['response'][a]['statistics'][1]['statistics'][4]['value'])
        shots_outsidebox_local.append(response['response'][a]['statistics'][0]['statistics'][5]['value'])
        shots_outsidebox_away.append(response['response'][a]['statistics'][1]['statistics'][5]['value'])
        fouls_local.append(response['response'][a]['statistics'][0]['statistics'][6]['value'])
        fouls_away.append(response['response'][a]['statistics'][1]['statistics'][6]['value'])
        corners_local.append(response['response'][a]['statistics'][0]['statistics'][7]['value'])
        corners_away.append(response['response'][a]['statistics'][1]['statistics'][7]['value'])
        offsides_local.append(response['response'][a]['statistics'][0]['statistics'][8]['value'])
        offsides_away.append(response['response'][a]['statistics'][1]['statistics'][8]['value'])
        ball_possession_local.append(response['response'][a]['statistics'][0]['statistics'][9]['value'])
        ball_possession_away.append(response['response'][a]['statistics'][1]['statistics'][9]['value'])
        yellow_cards_local.append(response['response'][a]['statistics'][0]['statistics'][10]['value'])
        yellow_cards_away.append(response['response'][a]['statistics'][1]['statistics'][10]['value'])
        red_cards_local.append(response['response'][a]['statistics'][0]['statistics'][11]['value'])
        red_cards_away.append(response['response'][a]['statistics'][1]['statistics'][11]['value'])
        goalkeeper_saves_local.append(response['response'][a]['statistics'][0]['statistics'][12]['value'])
        goalkeeper_saves_away.append(response['response'][a]['statistics'][1]['statistics'][12]['value'])
        pass_precision_local.append(response['response'][a]['statistics'][0]['statistics'][13]['value'])
        pass_precision_away.append(response['response'][a]['statistics'][1]['statistics'][13]['value'])

        time.sleep(0.21)
        flag+=1

In [984]:
#Construyo el DataFrame con las estadísticas de los partidos de la jornada

match_statistics_general = pd.DataFrame({'fixture_id': fixture_id_partido,
                                        'goles_local':goles_local,
                                        'id_local':id_local,
                                         'logo_local': logo_local,
                                         'logo_away':logo_away,
                                        'nombre_team_local': nombre_team_local,
                                        'goles_local_descanso': goles_local_descanso,
                                        'shots_on_goal_local':shots_on_goal_local,
                                        'shots_off_goal_local': shots_off_goal_local,
                                        'total_shots_local': total_shots_local,
                                        'blocked_shots_local': blocked_shots_local,
                                        'shots_insidebox_local': shots_insidebox_local,
                                        'shots_outsidebox_local': shots_outsidebox_local,
                                        'fouls_local': fouls_local,
                                        'corners_local': corners_local,
                                        'offsides_local': offsides_local,
                                        'ball_possession_local': ball_possession_local,
                                        'yellow_cards_local': yellow_cards_local,
                                        'red_cards_local': red_cards_local,
                                        'goalkeeper_saves_local': goalkeeper_saves_local,
                                        'pass_precision_local': pass_precision_local,
                                        'goles_away':goles_away,                                        
                                        'id_away':id_away,                                        
                                        'nombre_team_away': nombre_team_away,                                        
                                        'goles_away_descanso': goles_away_descanso,                                        
                                        'shots_on_goal_away': shots_on_goal_away,                                        
                                        'shots_off_goal_away': shots_off_goal_away,                                        
                                        'total_shots_away': total_shots_away,                                        
                                        'blocked_shots_away': blocked_shots_away,                                        
                                        'shots_insidebox_away': shots_insidebox_away,                                        
                                        'shots_outsidebox_away': shots_outsidebox_away,                                    
                                        'fouls_away': fouls_away,                                        
                                        'corners_away': corners_away,                                        
                                        'offsides_away': offsides_away,                                    
                                        'ball_possession_away': ball_possession_away,                                    
                                        'yellow_cards_away': yellow_cards_away,                                        
                                        'red_cards_away': red_cards_away,                                        
                                        'goalkeeper_saves_away': goalkeeper_saves_away,
                                        'pass_precision_away': pass_precision_away
                                        })

match_statistics_general.fillna(0, inplace=True)

match_statistics_general.to_sql('stats_partidos_general',conn, if_exists='replace')


964

In [978]:
#Construyo el DataFrame con las estadísticas de los equipos por jornada

matchs_statistics_team = pd.DataFrame({'fixture_id':fx_id,
                                        'jornada': jornada,
                                       'puntos_conseguidos' : puntos,
                                        'goles_anotados': goles_a_favor,
                                        'id_equipo':id_equipo,
                                       'nombre_team': nombre_team,
                                       'id_equipo_rival' : id_equipo_rival,
                                        'goles_marcados_descanso':goles_a_favor_descanso,
                                        'tiros_puerta': shots_on_goal,
                                        'tiros_fuera':shots_off_goal,
                                        'total_tiros':total_shots,
                                        'tiros_bloqueados': blocked_shots,
                                        'tiros_dentro_area': shots_insidebox,
                                        'tiros_fuera_area':shots_outsidebox,
                                        'faltas':fouls,
                                        'corners':corners,
                                        'fueras_juego':offsides,
                                        'posesion':ball_possession,
                                        'tarjetas_amarillas':yellow_cards,
                                        'tarjetas_rojas': red_cards,
                                        'paradas_portero_rival':goalkeeper_saves,
                                        'precision_pases':pass_precision,
                                        'condicion':condicion})


matchs_statistics_team.fillna(0, inplace=True)

matchs_statistics_team.to_sql('stats_partidos_team',conn, if_exists='replace')


1928

In [979]:
#Construyo el DataFrame con las estadísticas de los equipos en contra por jornada (lo que le hacen los rivales)

matchs_statistics_rival = pd.DataFrame({'fixture_id':fx_id,
                                        'jornada': jornada,
                                        'puntos_perdidos': puntos_perdidos,
                                        'goles_recibidos': goles_recibidos,
                                        'id_equipo':id_equipo,
                                        'nombre_team' : nombre_team,
                                        'goles_recibidos_descanso':goles_recibidos_descanso,
                                        'tiros_puerta_rival': shots_on_goal_rival,
                                        'tiros_fuera_rival':shots_off_goal_rival,
                                        'total_tiros_rival':total_shots_rival,
                                        'tiros_bloqueados_rival': blocked_shots_rival,
                                        'tiros_dentro_area_rival': shots_insidebox_rival,
                                        'tiros_fuera_area_rival':shots_outsidebox_rival,
                                        'faltas_rival':fouls_rival,
                                        'corners_rival':corners_rival,
                                        'fueras_juego_rival':offsides_rival,
                                        'posesion_rival':ball_possession_rival,
                                        'tarjetas_amarillas_rival':yellow_cards_rival,
                                        'tarjetas_rojas_rival': red_cards_rival,
                                        'paradas_portero_rival':goalkeeper_saves_rival,
                                        'precision_pases_rival':pass_precision_rival,
                                       'condición': condicion})


matchs_statistics_rival.fillna(0, inplace=True)

matchs_statistics_rival.to_sql('stats_partidos_rival',conn, if_exists='replace')

1928

In [ ]:
#Extracción de la clasificación de las ligas utilizadas (la usaré más adelante para poder 
#vincular los datos de transfermaket con los datos de la api)

puesto_clasif = []
nombre_liga_clasif = []
nombre_equipo_clasif = []
id_equipo_clasif=[]
logo_liga = []

for liga in id_ligas_top:

    url = "https://api-football-v1.p.rapidapi.com/v3/standings"

    querystring = {"season":"2022","league": liga}

    headers = {
        "X-RapidAPI-Key": "b78f5cd68cmsh17c6227d104p1755afjsn96dc33575295",
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring).json()

    for a in range(len(response['response'][0]['league']['standings'][0])):
        puesto_clasif.append(response['response'][0]['league']['standings'][0][a]['rank'])
        nombre_liga_clasif.append(response['response'][0]['league']['name'])
        logo_liga.append(response['response'][0]['league']['logo'])
        nombre_equipo_clasif.append(response['response'][0]['league']['standings'][0][a]['team']['name'])
        id_equipo_clasif.append(response['response'][0]['league']['standings'][0][a]['team']['id'])





In [918]:
#Creación del DataFrame de la clasificación de la api

clasificacion_api = pd.DataFrame({'posicion': puesto_clasif,
                                 'nombre_liga': nombre_liga_clasif,
                                  'logo_liga':logo_liga,
                                 'nombre_equipo': nombre_equipo_clasif,
                                 'id_equipo': id_equipo_clasif
                                })


In [919]:
#Extraigo el nombre de las ligas para poder unir a través de la clasificación de la api y
#de transfermarket los datos de cada equipo con su valor de mercado.

nombre_liga_clasif_unico = list(set(nombre_liga_clasif))
nombre_liga_clasif_unico

['La Liga',
 'Serie A',
 'Bundesliga',
 'Ligue 1',
 'Primeira Liga',
 'Premier League',
 'Eredivisie']

In [936]:
#Scrapeo de las distintas clasificaciones (en orden en función de la celda anterior) 
#y valor de los equipos de las ligas escogidas para el EDA

import requests

url = [
        'https://www.transfermarkt.es/laliga/startseite/wettbewerb/ES1',
        'https://www.transfermarkt.es/serie-a/startseite/wettbewerb/IT1',
        'https://www.transfermarkt.es/bundesliga/startseite/wettbewerb/L1',       
        'https://www.transfermarkt.es/ligue-1/startseite/wettbewerb/FR1',
        'https://www.transfermarkt.es/liga-nos/startseite/wettbewerb/PO1',
        'https://www.transfermarkt.es/premier-league/startseite/wettbewerb/GB1',
        'https://www.transfermarkt.es/eredivisie/startseite/wettbewerb/NL1'
      ]

tablas_clasif = []
valor_mercado = []

for x in url:
    tablas_clasif.append((pd.read_html(requests.get(x, 
                                                    headers={'User-agent': 'Mozilla/5.0'}).text, 
                                                    attrs={"class":"items"})[3]))
    valor_mercado.append((pd.read_html(requests.get(x, 
                                                    headers={'User-agent': 'Mozilla/5.0'}).text, 
                                                    attrs={"class":"items"})[0]))
        

In [937]:
#Para el cambio de moneda:

def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    elif '.' in x:
        if len(x) > 1:
            return float(x.replace(' mill. €', '').replace(',','.')) * 1000000
        return 1000.0
    elif 'mil ' in x:
        if len(x) > 1:
            return float(x.replace(' mil €', '').replace(',','.')) * 1000
        return 1000000.0


In [938]:
#Limpieza de la tabla de valor de mercado y seleccion de datos interesantes. El Manchester City me entraba en conflicto porque 
#es el único equipo que su valor es superior a los 1000 millones.

for tabla in valor_mercado:
    tabla.drop('Club', axis=1, inplace=True)
    tabla.drop('Equipo', axis=1, inplace=True)
    tabla.drop('name', axis=1, inplace=True)
    tabla.drop(index=tabla.index[-1], axis=0, inplace=True)
    tabla.drop('Valor de mercado total', axis=1, inplace=True)
    tabla.rename(columns={'Club.1': 'equipo',
                          'Equipo' : 'edad_media',
                          'Edad' : 'no_extranjeros',
                          'Extranjeros':'media_valor_mercado_jugador',
                          'ø-Valor de mercado': 'valor_mercado_total'
                         }, inplace=True)
    for index, row in tabla.iterrows():    
        if row['equipo'] != 'Manchester City':
            row['media_valor_mercado_jugador'] = value_to_float(row['media_valor_mercado_jugador'])
            row['valor_mercado_total'] = value_to_float(row['valor_mercado_total'])
        if row['equipo'] == 'Manchester City':
            row['media_valor_mercado_jugador'] = float(row['media_valor_mercado_jugador'].replace(' mill. €', '').replace(',','.'))*1000000
            row['valor_mercado_total'] = float(row['valor_mercado_total'].replace(' mil mill. €', '').replace(',','.'))*1000000000

In [939]:
#Limpieza de la tabla de clasificación y seleccion de datos interesantes
flag=0
for tabla in tablas_clasif:
    tabla.drop('Club', axis=1, inplace=True)
    tabla.rename(columns={'#':'clasificacion', 
                            'Club.1':'equipo', 
                            'Unnamed: 3': 'partidos_jugados',
                            '+/-':'diferencia_goles',
                            'Pto.':'puntos'},
                    inplace=True)
    tabla.drop('diferencia_goles', axis=1, inplace=True)
    tabla['liga'] = nombre_liga_clasif_unico[flag]
    flag+=1

In [941]:
#Uno todas las clasificaciones de transfermakt en una sola tabla
valor_mercado_tabla= pd.concat([  valor_mercado[0],
                                   valor_mercado[1],
                                   valor_mercado[2],
                                   valor_mercado[3],
                                   valor_mercado[4],
                                   valor_mercado[5],
                                   valor_mercado[6],
                                   ])

In [942]:
#Uno todas las clasificaciones de transfermakt en una sola tabla
clasif_transfermarkt= pd.concat([  tablas_clasif[0],
                                   tablas_clasif[1],
                                   tablas_clasif[2],
                                   tablas_clasif[3],
                                   tablas_clasif[4],
                                   tablas_clasif[5],
                                   tablas_clasif[6],
                                   ])

In [943]:
#Cambio los nombres de los equipos de la clasificacion para poder unirlos con la tabla de valor de equipos
clasif_transfermarkt['equipo'] = clasif_transfermarkt['equipo'].map({'Real Madrid': 'Real Madrid CF', 
                                                                     'Athletic': 'Athletic Club',
                                                                    'Real Sociedad' : 'Real Sociedad',
                                                                    'FC Barcelona':'FC Barcelona',
                                                                   'Atlético Madrid':'Atlético de Madrid',
                                                                    'CA Osasuna':'CA Osasuna',
                                                                    'Rayo Vallecano': 'Rayo Vallecano',
                                                                    'Villarreal CF': 'Villarreal CF',
                                                                    'Valencia CF': 'Valencia CF',
                                                                    'RCD Mallorca': 'RCD Mallorca',
                                                                    'Girona FC': 'Girona FC',
                                                                    'UD Almería':'UD Almería',
                                                                    'RCD Espanyol': 'RCD Espanyol',
                                                                    'Sevilla FC': 'Sevilla FC',
                                                                    'Cádiz CF': 'Cádiz CF',
                                                                    'Elche CF': 'Elche CF',
                                                                    'Real Betis': 'Real Betis Balompié',
                                                                    'Getafe': 'Getafe CF',
                                                                    'RC Celta': 'RC Celta de Vigo',
                                                                    'Real Valladolid':'Real Valladolid CF',
                                                                     'Nápoles' : 'SSC Nápoles',
                                                                     'AC Milan': 'AC Milan',
                                                                     'Juventus':'Juventus de Turín',
                                                                     'Lazio' : 'SS Lazio',
                                                                     'Inter' : 'Inter de Milán',
                                                                     'Atalanta' : 'Atalanta de Bérgamo',
                                                                     'AS Roma': 'AS Roma',
                                                                     'Udinese': 'Udinese',
                                                                     'Torino': 'Torino FC',
                                                                     'Fiorentina' : 'Fiorentina',
                                                                     'Bolonia': 'Bolonia',
                                                                     'Salernitana': 'US Salernitana 1919',
                                                                     'FC Empoli':'FC Empoli',
                                                                     'Monza' : 'AC Monza',
                                                                     'Sassuolo':'US Sassuolo',
                                                                     'Lecce':'US Lecce',
                                                                     'Spezia':'Spezia',
                                                                     'Cremonese': 'US Cremonese',
                                                                     'Sampdoria':'UC Sampdoria',
                                                                     'Hellas Verona':'Hellas Verona',
                                                                     'Bayern Múnich':'Bayern Múnich',
                                                                     'RB Leipzig':'RB Leipzig',
                                                                     'Unión Berlín':'1.FC Unión Berlín',
                                                                     'SC Friburgo':'SC Friburgo',
                                                                     'Eintracht': 'Eintracht Fráncfort​​',
                                                                     'Bor. Dortmund':'Borussia Dortmund',
                                                                     "Bor. M'gladbach": 'Borussia Mönchengladbach',
                                                                     'VfL Wolfsburgo':'VfL Wolfsburgo',
                                                                     'Werder Bremen':'SV Werder Bremen',
                                                                     'Hoffenheim': 'TSG 1899 Hoffenheim',
                                                                     'Mainz':'1.FSV Mainz 05',
                                                                     'Colonia':'FC Colonia',
                                                                     'B. Leverkusen': 'Bayer 04 Leverkusen',
                                                                     'FC Augsburgo':'FC Augsburgo',
                                                                     'VfB Stuttgart':'VfB Stuttgart',
                                                                     'Hertha Berlín':'Hertha Berlín',
                                                                     'VfL Bochum': 'VfL Bochum',
                                                                     'FC Schalke 04':'FC Schalke 04',
                                                                     'PSG':'París Saint-Germain FC',
                                                                     'Lens':'RC Lens',
                                                                     'Rennes': 'Stade Rennais FC',
                                                                     'Marsella': 'Olympique de Marsella',
                                                                     'FC Lorient':'FC Lorient',
                                                                     'Mónaco':'AS Mónaco',
                                                                     'Lille':'LOSC Lille Métropole',
                                                                     'Lyon':'Olympique de Lyon',
                                                                     'Niza':'OGC Niza',
                                                                     'Clermont Foot':'Clermont Foot 63',
                                                                     'Stade Reims':'Stade Reims',
                                                                     'Toulouse':'Toulouse FC',
                                                                     'Troyes':'ESTAC Troyes',
                                                                     'Montpellier':'Montpellier HSC',
                                                                     'FC Nantes':'FC Nantes',
                                                                     'Stade Brestois':'Stade Brestois 29',
                                                                     'AJ Auxerre':'AJ Auxerre',
                                                                     'AC Ajaccio':'AC Ajaccio',
                                                                     'Estrasburgo':'Racing Club de Estrasburgo',
                                                                     'Angers SCO':'Angers SCO',
                                                                     'Arsenal':'Arsenal FC',
                                                                     'Manchester City':'Manchester City',
                                                                    'Newcastle Utd.':'Newcastle United',
                                                                     'Tottenham':'Tottenham Hotspur',
                                                                     'Manchester Utd.':'Manchester United',
                                                                     'Liverpool':'Liverpool FC',
                                                                     'Brighton':'Brighton & Hove Albion',
                                                                     'Chelsea':'Chelsea FC',
                                                                     'Fulham': 'Fulham FC',
                                                                     'Brentford':'Brentford FC',
                                                                     'Crystal Palace':'Crystal Palace',
                                                                     'Aston Villa':'Aston Villa',
                                                                     'Leicester City':'Leicester City',
                                                                     'Bournemouth':'AFC Bournemouth',
                                                                     'Leeds': 'Leeds United',
                                                                     'West Ham':'West Ham United',
                                                                     'Everton':'Everton FC',
                                                                     'Nottm Forest':'Nottingham Forest',
                                                                     'Southampton':'Southampton FC',
                                                                     'Wolverhampton':'Wolverhampton Wanderers',
                                                                     'Benfica':'SL Benfica',
                                                                     'Oporto':'FC Oporto',
                                                                     'Sporting CP': 'Sporting CP',
                                                                     'SC Braga':'SC Braga',
                                                                     'Casa Pia':'Casa Pia AC',
                                                                     'Vit. Guimarães':'Vitória Guimarães SC',
                                                                     'Portimonense':'Portimonense SC',
                                                                     'Chaves':'GD Chaves',
                                                                     'Arouca':'FC Arouca',
                                                                     'Rio Ave FC':'Rio Ave FC',
                                                                     'Boavista':'Boavista Porto FC',
                                                                     'Estoril Praia':'GD Estoril Praia',
                                                                     'Vizela':'FC Vizela',
                                                                     'Santa Clara':'CD Santa Clara',
                                                                     'Famalicão':'FC Famalicão',
                                                                     'Gil Vicente':'Gil Vicente FC',
                                                                     'Marítimo':'CS Marítimo',
                                                                     'Paços Ferreira':'FC Paços de Ferreira',
                                                                     'Feyenoord':'Feyenoord Rotterdam',
                                                                     'Ajax':'Ajax de Ámsterdam',
                                                                     'PSV Eindhoven':'PSV Eindhoven',
                                                                     'AZ Alkmaar':'AZ Alkmaar',
                                                                     'FC Twente':'FC Twente Enschede',
                                                                     'Sparta R.':'Sparta Rotterdam',
                                                                     'FC Utrecht':'FC Utrecht',
                                                                     'Heerenveen':'SC Heerenveen',
                                                                     'NEC Nijmegen':'NEC Nijmegen',
                                                                     'RKC Waalwijk':'RKC Waalwijk',
                                                                     'Go Ahead Eagles':'Go Ahead Eagles Deventer',
                                                                     'Fortuna Sittard':'Fortuna Sittard',
                                                                     'Vitesse':'Vitesse Arnhem',
                                                                     'Excelsior':'Excelsior Rotterdam',
                                                                     'FC Groningen':'FC Groningen',
                                                                     'FC Emmen':'FC Emmen',
                                                                     'SC Cambuur':'SC Cambuur-Leeuwarden',
                                                                     'FC Volendam':'FC Volendam'
                                                                    })

In [945]:
#Creo una columna con un id_posicion_liga para poder hacer el merge correctamente con ids únicos

clasif_transfermarkt['id_pos_liga'] = clasif_transfermarkt['clasificacion'].astype('string')+'-'+clasif_transfermarkt['liga'].astype('string')
clasificacion_api['id_pos_liga'] = clasificacion_api['posicion'].astype('string')+'-'+clasificacion_api['nombre_liga'].astype('string')

In [946]:
#junto las dos clasificaciones en un merge para unificar los nombres de los equipos bajo un mismo id_equipo

clasificacion_equipos = clasificacion_api.merge(clasif_transfermarkt)


In [948]:
#añado la tabla de clasificaciones a la BBDD 

clasificacion_equipos.to_sql('clasificacion_equipos',conn, if_exists='replace')

134

In [949]:
#añado la tabla de valor de mercado a la BBDD 

valor_mercado_tabla.to_sql('valor_mercado_equipos', conn, if_exists='replace')

134